In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import grid_search
from sklearn import cross_validation

import seaborn as sns
sns.set_style('whitegrid') 

In [5]:
from plot_roc_with_cv import *
%load_ext autoreload
%autoreload 2

None


In [4]:
saveas_path = '/Users/jamie/insight_data/figures'

In [6]:
%matplotlib qt

In [23]:
plot_roc_with_cv()

Built-in functions, exceptions, and other objects.

Noteworthy: None is the `nil' object; Ellipsis represents `...' in slices.
(115196,)


In [27]:
fig = plt.gcf()
fig.set_size_inches(6,6)

In [29]:
plt.title('Receiver operating characteristic')

In [30]:
plt.savefig(saveas_path + 'random forests roc with cv.png',bbox_inches='tight',dpi=400) 

In [7]:
X_df = pd.read_pickle('new_X_df.save')
outcomes_df = pd.read_pickle('new_outcomes_df.save')

In [9]:
X_train, X_test, y_train, y_test = \
    cross_validation.train_test_split(X_df, outcomes_df['Outcome'].values, \
                                      test_size=0.25, random_state=0)

In [14]:
# from sklearn.pipeline import Pipeline
# from sklearn.svm import LinearSVC

# clf = Pipeline([
#   ('feature_selection', LinearSVC()),
#   ('classification', RandomForestClassifier())
# ])
# clf.fit(X_train, y_train)

Pipeline(steps=[('feature_selection', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)), ('classification', RandomForestClassifier(bootstrap=Tr...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [42]:
# parameters here -http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=600, max_depth=20, n_jobs=-1)
clf.fit(X_train,y_train)
y_1 = clf.predict(X_test)

In [43]:
print 'accuracy = ', round(clf.score(X_test,y_test),4)

accuracy =  0.7665


In [44]:
metrics.roc_auc_score(y_test, y_1)

0.75674133632712559

In [45]:
from sklearn import metrics
metrics.precision_recall_fscore_support(y_test,y_1)

(array([ 0.79404796,  0.72507395]),
 array([ 0.81302881,  0.70045386]),
 array([ 0.8034263,  0.7125513]),
 array([16901, 11898]))

In [66]:
rf = RandomForestClassifier()
# parameters = {'n_estimators':[100,200,400],
#               'max_depth':[5,7]}

parameters = {'n_estimators':[100,200,300,400,600,800],
              'max_depth':[5,6,7,8,9,10,12,14,16,18,20,22,24]}

In [69]:
clf_grid = grid_search.GridSearchCV(rf,param_grid=parameters,\
                            scoring='roc_auc',n_jobs=-1, verbose=1)

In [70]:
clf_grid.fit(X_train, y_train)

Fitting 3 folds for each of 78 candidates, totalling 234 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 200 jobs       | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 234 out of 234 | elapsed: 24.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid={'n_estimators': [100, 200, 300, 400, 600, 800], 'max_depth': [5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 22, 24]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='roc_auc', verbose=1)

In [101]:
# for score in clf_grid.grid_scores_:
#     print score

mean: 0.81104, std: 0.00149, params: {'n_estimators': 100, 'max_depth': 5}
mean: 0.81333, std: 0.00067, params: {'n_estimators': 200, 'max_depth': 5}
mean: 0.81265, std: 0.00060, params: {'n_estimators': 300, 'max_depth': 5}
mean: 0.81205, std: 0.00023, params: {'n_estimators': 400, 'max_depth': 5}
mean: 0.81101, std: 0.00059, params: {'n_estimators': 600, 'max_depth': 5}
mean: 0.81076, std: 0.00066, params: {'n_estimators': 800, 'max_depth': 5}
mean: 0.81719, std: 0.00085, params: {'n_estimators': 100, 'max_depth': 6}
mean: 0.81655, std: 0.00084, params: {'n_estimators': 200, 'max_depth': 6}
mean: 0.81730, std: 0.00071, params: {'n_estimators': 300, 'max_depth': 6}
mean: 0.81859, std: 0.00098, params: {'n_estimators': 400, 'max_depth': 6}
mean: 0.81929, std: 0.00105, params: {'n_estimators': 600, 'max_depth': 6}
mean: 0.81869, std: 0.00102, params: {'n_estimators': 800, 'max_depth': 6}
mean: 0.82454, std: 0.00079, params: {'n_estimators': 100, 'max_depth': 7}
mean: 0.82392, std: 0.000

In [87]:
clf_grid.best_params_

{'max_depth': 22, 'n_estimators': 800}

In [88]:
clf_grid.best_score_

0.8459956407894188

In [89]:
y_1 = clf_grid.predict(X_test)

In [96]:
rf_best = clf_grid.best_estimator_

In [97]:
y_1 = clf_grid.predict(X_test)

In [98]:
metrics.roc_auc_score(y_test, y_1) # average='macro',

0.75796337533611757

In [99]:
metrics.precision_recall_fscore_support(y_test, y_1)

(array([ 0.794974 ,  0.7266951]),
 array([ 0.81421218,  0.70171457]),
 array([ 0.80447809,  0.7139864 ]),
 array([16901, 11898]))

In [100]:
print 'accuracy = ', round(rf_best.score(X_test,y_test),4)

accuracy =  0.7677


In [92]:
#plotROCCurve(y_test, clf_grid.predict(X_test))

In [93]:
#from sklearn.externals import joblib
#joblib.dump(clf_grid, 'clf_grid_june24am.pkl') 